## MLB data and API for forecasting
Starting to get the data
From the documentation and exemple<br>
https://www.kaggle.com/chumajin/eda-of-mlb-for-starter-english-ver<br>
https://www.kaggle.com/ryanholbrook/getting-started-with-mlb-player-digital-engagement<br>
-I build 2 pickles files from the train.csv file

In [ ]:
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
from pathlib import Path
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error

import pickle

from IPython.display import Image


In [ ]:

Image("../input/pictures/johnny_automatic_old_time_pitcher.jpg")

In [ ]:
with open("../input/pickle-data/df_nextDayPlayer.pkl", 'rb') as handle:
    df_nextDayPlayer = pickle.load(handle)
    
with open("../input/pickle-data/playerBoxScores.pkl", 'rb') as handle:
    df_playerBoxScores = pickle.load(handle)

In [ ]:
df_playerBoxScores.head(n=2)

In [ ]:
df_nextDayPlayer.head(n=2)

In [ ]:
print(df_playerBoxScores.shape)
print((df_nextDayPlayer.shape))

In [ ]:
#--------------------------prepare data sets-------------------------------
df_nextDayPlayer.nunique()

# Data Player Box Scores

In [ ]:
df_playerBoxScores.info()

In [ ]:
targets = ["target1", "target2", "target3", "target4"]
features = [
    "hits",
    'doubles',
    'triples',
    "strikeOuts",
    "homeRuns",
    'atBats',
    "runsScored",
    "stolenBases",
    'caughtStealing',
    "strikeOutsPitching",
    "inningsPitched",
    "strikes",
    "flyOuts",
    "groundOuts",
    "errors",
    "chances",
    'TotalBase',
    'BattingAvg',
    
]

In [ ]:
#----------------------adding calcul columns----------------------------
#Adding Total bats columns
df_playerBoxScores['TotalBase']=df_playerBoxScores['hits']+(2*df_playerBoxScores['doubles'])+(3*df_playerBoxScores['triples'])+(4*df_playerBoxScores['homeRuns'])
#Adding Batting Average
df_playerBoxScores['BattingAvg']=df_playerBoxScores['hits']/df_playerBoxScores['atBats']

In [ ]:
#----------prepare data player Box scores----------------------------------
df_playerBoxScores.rename(columns={"gameDate": "date"}, inplace=True)
df_playerBoxScores = df_playerBoxScores[['date', 'playerId'] + features]
# Set dtypes
df_playerBoxScores = df_playerBoxScores.astype({name: np.float32 for name in features})
df_playerBoxScores = df_playerBoxScores.astype({'playerId': str})
df_playerBoxScores.groupby(['date', 'playerId'], as_index=False).sum()

# exploring data with some player Id

In [ ]:
#Data seasonalite----------------------------------------------------------------
season_Global_2020=df_playerBoxScores[df_playerBoxScores.date.between('2018-01-02', '2020-12-31')]
season_Global_2020=season_Global_2020.sort_values(by='BattingAvg', ascending=False)
season_Global_2020=season_Global_2020.sort_values(by='date')
season_Global_2020.groupby(['playerId', 'date'], axis=0).sum()

In [ ]:
season_Global_2020_02=season_Global_2020[season_Global_2020['playerId']=="543105"]

In [ ]:
#ploting some performances
fig, ax = plt.subplots(figsize=(24,7))
ax.plot(season_Global_2020_02.date,season_Global_2020_02.BattingAvg, label='BattingAvg', fillstyle="full", linewidth=2 )
ax.plot(season_Global_2020_02.date,season_Global_2020_02.homeRuns, label='homeRuns', fillstyle="full", linewidth=2 )
plt.xlabel("date", labelpad=10)
plt.title('playerId 543105')
#plt.xticks(rotation=45)
plt.xticks([])
plt.legend(frameon=False)
plt.show()


In [ ]:
#ploting some performances
fig, ax = plt.subplots(figsize=(24,7))
ax.plot(season_Global_2020_02.date,season_Global_2020_02.runsScored,label='runsScored', fillstyle="full", linewidth=2 )
plt.xlabel("date", labelpad=10)
plt.title('playerId 543105')
#plt.xticks(rotation=45)
plt.xticks([])
plt.legend(frameon=False)
plt.show()

<p>Next step is to prepare next day player to build a global data frame</p>


In [ ]:
Image("../input/pictures/johnny_automatic_baseball_at_bat.jpg")

# Data Next Day Player

In [ ]:
#---------------Prepare next Day Player data------------------------------------------------------------------------------------
df_nextDayPlayer.rename(columns={"engagementMetricsDate": "date"}, inplace=True)
df_nextDayPlayer = df_nextDayPlayer.astype({name: np.float32 for name in targets})
df_nextDayPlayer = df_nextDayPlayer.astype({'playerId': str})
df_nextDayPlayer=df_nextDayPlayer.groupby(['date', 'playerId'], as_index=False).sum()

In [ ]:
df_nextDayPlayer.head()

In [ ]:

#----------------------------------------------Data seasonalite----------------------------------------------------------------
season_nextdayPlayer_2020=df_nextDayPlayer[df_nextDayPlayer.date.between('2019-01-01', '2021-12-31')]
season_nextdayPlayer_2020=season_nextdayPlayer_2020.sort_values(by='target1', ascending=False)
season_nextdayPlayer_2020=season_nextdayPlayer_2020.sort_values(by='date')


In [ ]:
season_nextdayPlayer_02=season_nextdayPlayer_2020[(season_nextdayPlayer_2020['playerId']=="543105")]
season_nextdayPlayer_03=season_nextdayPlayer_2020[(season_nextdayPlayer_2020['playerId']=="282332")]
season_nextdayPlayer_04=season_nextdayPlayer_2020[(season_nextdayPlayer_2020['playerId']=="516969")]

# Understand the correlation between targets with differents player Id

In [ ]:
#Heatmap1 player Id : 543105
plt.figure(figsize=(17,11))
#heatmap1
corr = season_nextdayPlayer_02.corr()
cmap = sns.diverging_palette(230, 20, as_cmap=True)
plt.subplot(1, 3, 1)
sns.heatmap(corr, center=0,annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Heatmap player Id : 543105')
#heatmap2 player Id : 282332
corr02 = season_nextdayPlayer_04.corr()
plt.subplot(1, 3, 2)
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr02, center=0,annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Heatmap player Id : 282332')
#heatmap3 player Id : 516969
corr03 = season_nextdayPlayer_03.corr()
plt.subplot(1, 3, 3)
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr03, center=0,annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Heatmap player Id : 516969')

plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(24,7))
ax.plot(season_nextdayPlayer_02.date,season_nextdayPlayer_02.target1, label='target1', fillstyle="full", linewidth=2 )
ax.plot(season_nextdayPlayer_02.date,season_nextdayPlayer_02.target2, label='target2', fillstyle="full", linewidth=2 )
ax.plot(season_nextdayPlayer_02.date,season_nextdayPlayer_02.target3, label='target3', fillstyle="full", linewidth=2 )
ax.plot(season_nextdayPlayer_02.date,season_nextdayPlayer_02.target4, label='target4', fillstyle="full", linewidth=2 )
plt.xlabel("date", labelpad=10)
plt.title('playerId 543105 and targets')
plt.xticks([])
plt.legend(frameon=False)
plt.show()

In [ ]:
season_nextdayPlayer_02["target1"][-30:].plot(figsize=(12,4),label='target1', fillstyle="full", linewidth=2);
season_nextdayPlayer_02["target2"][-30:].plot(figsize=(12,4),label='target2', fillstyle="full", linewidth=2);
season_nextdayPlayer_02["target3"][-30:].plot(figsize=(12,4),label='target3', fillstyle="full", linewidth=2);
season_nextdayPlayer_02["target4"][-30:].plot(figsize=(12,4),label='target4', fillstyle="full", linewidth=2);
plt.title('playerId 543105 and targets')
plt.legend(frameon=False)
plt.show()

<p>Making some graph to understand the way he seasonality can be different for player or season</p>

In [ ]:
from statsmodels.tsa.seasonal import STL
#function for trend seasonal and resid plot
def add_stl_plot(fig, res, legend):
    axs = fig.get_axes()
    comps = ['trend', 'seasonal', 'resid']
    for ax, comp in zip(axs[1:], comps):
        series = getattr(res, comp)
        if comp == 'resid':
            ax.plot(series, marker='o', linestyle='none')
        else:
            ax.plot(series)
            if comp == 'trend':
                ax.legend(legend, frameon=False)

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set_style('darkgrid')
plt.rc('figure',figsize=(16,12))
plt.rc('font',size=13)
plt.figure(figsize=(17,11))
stl = STL(season_nextdayPlayer_02.target1, period=7, robust=True)
res_robust = stl.fit()
fig = res_robust.plot()
res_non_robust = STL(season_nextdayPlayer_02.target1, period=7, robust=False).fit()
add_stl_plot(fig, res_non_robust, ['Robust','Non-robust'])
plt.title('playerId 543105 seasonal trend and resid')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(season_nextdayPlayer_02["target2"][-395:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = plot_pacf(season_nextdayPlayer_02["target2"][-395:], lags=40, ax=ax2);

In [ ]:
#Using Regular season start and end date
season_OnedayPlayer_2020=df_nextDayPlayer[df_nextDayPlayer.date.between('2021-02-28', '2021-10-31')]
#exeample for one player 
season_OnedayPlayer_2020=season_OnedayPlayer_2020[(season_OnedayPlayer_2020['playerId']=="282332")]

In [ ]:
season_OnedayPlayer_2020.isnull().sum()

In [ ]:
series = np.log(season_OnedayPlayer_2020["target2"])
series01 = np.log(season_OnedayPlayer_2020["target1"])
series03 = np.log(season_OnedayPlayer_2020["target3"])
series04 = np.log(season_OnedayPlayer_2020["target4"])

plt.figure(figsize=(12,8))
plt.plot(series, label="Series target 2")
plt.plot(series01, label="Series target 1")
plt.plot(series03, label="Series target 3")
plt.plot(series04, label="Series target 4")
plt.legend()
plt.show()

## SARIMA for one player for season 2020

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

size = int(len(season_OnedayPlayer_2020['target2'].dropna()) * 0.75)
train, test = season_OnedayPlayer_2020['target2'].dropna()[0:size], season_OnedayPlayer_2020['target2'].dropna()[size:len(season_nextdayPlayer_02["target2"].dropna())]
test = test.reset_index()['target2']
history = [x for x in train]
predictions = list()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for t in range(len(test)):
    model = SARIMAX(history, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(yhat)

In [ ]:
history = [x for x in train]

plt.figure(figsize=(12,8))
plt.plot(np.concatenate([history, predictions]), label='Prediction')
plt.plot(np.concatenate([history, test]), label='Test')
plt.title('prediction for palyer Id 282332 and target2')
plt.legend()
plt.show()

In [ ]:
mean_squared_error(predictions, test)

# building a model with LSTM
# Using sequence of targets 

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics 
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from statsmodels.tsa.seasonal import STL

from math import sqrt

import tensorflow as tf
import keras
from keras import layers
from keras import Model
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Input, Dense, concatenate

In [ ]:
df_nextDayPlayer['date'] = pd.to_datetime(df_nextDayPlayer['date'], format="%Y-%m-%d")
df_nextDayPlayer = df_nextDayPlayer.set_index('date').to_period('D')
print(df_nextDayPlayer.info())

## Starting to make some preprocessing to make a train data
## and scaling values for supervised learning

In [ ]:
# convert series to supervised learning
from pandas import concat
from pandas import DataFrame
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 

In [ ]:
df_nextDayPlayer['playerId'].nunique()

In [ ]:
cols=['target1', 'target2', 'target3', 'target4']
#using scaling for sampling the trainning set  
df_train=df_nextDayPlayer[cols].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df_train)

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[2,3,4,5]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
values = reframed.values
seq_time = 365*2061
train = values[:seq_time, :]
test = values[seq_time:, :]

In [ ]:
# split into input and outputs
X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
# reshape input to be [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape) 

### Using LSTM model with inputs and outputs

In [ ]:
model = Sequential()
model.add(LSTM(100,input_shape=(X_train.shape[1], X_train.shape[2]),activation="relu", bias_initializer='zeros', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu',bias_initializer='zeros',return_sequences=True))
model.add(LSTM(10))
model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))
model.compile( optimizer='adam', loss='mean_squared_error', metrics='mae')

In [ ]:
history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


### Test prediction and inverse

In [ ]:
# plot history
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# Evaluation of the model
loss, mean_squared_error= model.evaluate(X_test, y_test, verbose=0)
print('loss is:', loss)
print('mean squared error is:', mean_squared_error)

In [ ]:
#save model 
model.save("model.h5")

In [ ]:
test_predict=model.predict(X_test)

In [ ]:
test_X = X_test.reshape((X_test.shape[0], X_test.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((test_predict, test_X[:, -4:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
inv_y = np.concatenate((y_test, test_X[:, -4:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
plt.figure(figsize=(12,8))
aa=[x for x in range(365)]
plt.plot(aa, inv_y[:365], marker='.', label="actual")
plt.plot(aa, inv_yhat[:365], 'r', label="prediction")
plt.ylabel('player activity', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()

### to make the submission i use the example of notebook
https://www.kaggle.com/ruriarmandhani/mlb-submission<br>
https://www.kaggle.com/ulrich07/mlb-debug-ann


In [ ]:
#-----------------------------Next day Player--------------------------------------
with open("../input/pickle-data/df_nextDayPlayer.pkl", 'rb') as handle:
    df_nextDayPlayer = pickle.load(handle)
    

In [ ]:
df_nextDayPlayer.rename(columns={"engagementMetricsDate": "date"}, inplace=True)
df_nextDayPlayer['date'] = pd.to_datetime(df_nextDayPlayer['date'], format="%Y-%m-%d")
df_nextDayPlayer= df_nextDayPlayer.reset_index()
#df_nextDayPlayer = df_nextDayPlayer.set_index('date').to_period('D')

### Sample data on mlb env

In [ ]:
import mlb

In [ ]:
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    
   
    #------sample_prediction-----
    sample_prediction_df= sample_prediction_df.reset_index()
    sample_prediction_df['date'] = pd.to_datetime(sample_prediction_df['date'], format='%Y%m%d')
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId'].apply(lambda x: x.split('_')[1]).astype(int)
    sample_prediction_df[targets]=df_nextDayPlayer[targets]
    sample_prediction_df = sample_prediction_df.fillna(0.)
    
     #----sample data---------------------------
    df_train=sample_prediction_df[targets].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(df_train)
    seq_time = 365*4
    test = scaled[:seq_time, :]
    train = scaled[seq_time:, :]
    X_train, y_train = train[:, :-1], train[:, -1]
    X_test, y_test = test[:, :-1], test[:, -1]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    
    #----make predict----
    model = keras.models.load_model("../input/modelplace/model .h5")
    pred= model.predict(X_test)
    
    
    #----sample for submission-----
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))
    pred = np.concatenate((pred, X_test[:, -4:]), axis=1)
    
    sample_prediction_df=sample_prediction_df.drop(columns=['date','playerId'])
    sample_prediction_df[targets] = np.clip(pred, 0, 100)
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df